In [49]:
import pandas as pd
import numpy as np
import re
import nltk
import sys
from nltk.corpus import stopwords

from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten
from keras.layers import GlobalMaxPooling1D
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Concatenate, Dense, Input, LSTM, Embedding, Dropout, Activation, GRU, Flatten
from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Convolution1D
from tensorflow.keras import initializers, regularizers, constraints, optimizers, layers
from tensorflow.keras.regularizers import l2

#import statments 

In [19]:
data = pd.read_csv('data_shuffled.csv')
data.head()

news = data['bodytext'].values
label = data['label'].values

news = np.delete(news, (4396), axis=0)
label = np.delete(label, (4396), axis=0)
label = np.array(label)

label = label.astype(np.float)



print(label)
print(type(label[2]))

In [ ]:
t = Tokenizer()
t.fit_on_texts(news)

text_matrix = t.texts_to_sequences(news)

print('News: ' + news[0])
print('Representation: ')
print(text_matrix[0])


In [47]:
t = Tokenizer()
t.fit_on_texts(news)

text_matrix = t.texts_to_sequences(news)

print('News: ' + news[0])
print('Representation: ')
print(text_matrix[0])

#find max length of vectors

max_length = 0
min_length = 1
total_length = 0

emb_dim = 16

vocab_size = len(t.word_index) + 1

for i in range(len(text_matrix)):
    sent_length = len(text_matrix[i])
    if max_length < sent_length:
        max_length = sent_length
        
for i in range(len(text_matrix)):
    sent_length = len(text_matrix[i])
    if min_length > sent_length:
        min_length = sent_length

for i in range(len(text_matrix)):
    sent_length = len(text_matrix[i])
    total_length += sent_length

print('Average Length %d' % int(total_length/19255))
print('vocabulary size: %d'%vocab_size)
print('Max Length: %d' % max_length)
print('Min Length: %d' % min_length)

print(type(text_matrix[1]))

Average Length 504
vocabulary size: 51000
Max Length: 5317
Min Length: 1


In [51]:
from tensorflow.keras.regularizers import l2



#split datasets 
#No need to post pad data since all have lengths 
text_matrix = pad_sequences(text_matrix, maxlen=max_length, padding='post')

# text_matrix = np.array(text_matrix, dtype=None)
# label = np.array(label, dtype= None)

X_train, X_test, y_train, y_test = train_test_split(text_matrix, label, test_size = 0.33, random_state = 53)

X_train = tf.ragged.constant(X_train)
X_test = tf.ragged.constant(X_test)
y_train = tf.ragged.constant(y_train)
y_test = tf.ragged.constant(y_test)

lstm_units = 12
inputs = Input(shape = (max_length,))
embedding = Embedding(input_dim=vocab_size, output_dim=emb_dim, input_length=max_length, embeddings_regularizer=l2(.001))
embd_out = embedding(inputs)

lstm = LSTM(lstm_units, dropout = 0.25, recurrent_dropout = 0.2)
lstm_out = lstm(embd_out)

prob = Dense(1, activation = 'relu')
outputs = prob(lstm_out)

model = Model(inputs, outputs)

print(model.summary())

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 5317)]            0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 5317, 16)          816000    
_________________________________________________________________
lstm (LSTM)                  (None, 12)                1392      
_________________________________________________________________
dense (Dense)                (None, 1)                 13        
Total params: 817,405
Trainable params: 817,405
Non-trainable params: 0
_________________________________________________________________
None


In [53]:

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

model.fit(x=X_train,y=y_train,
          batch_size=120,
          epochs=50,
          verbose=1,
          shuffle=True,
          validation_data=(X_test,y_test)
         )

KeyboardInterrupt: 